In [ ]:
import os
import time

import torch

from apt.model import APTClassifier
from apt.utils import process_data

In [ ]:
model = APTClassifier(device="cuda:0")

In [ ]:
save_dir = "results/"
for split in range(1, 6):
    data_dir = f"datasets/classification/OpenML-CC18-{split}"
    datasets = os.listdir(data_dir)
    datasets = [dataset for dataset in datasets if ".pt" in dataset]

    with open(os.path.join(save_dir, f"apt-classification-{split}.csv"), "a") as f:
        f.write(','.join(["dataset", "acc", "bacc", "ce", "auc", "time"]))
        f.write('\n')
        f.flush()
        for dataset in datasets:
            data = torch.load(os.path.join(data_dir, dataset), map_location='cpu')
            x_train, y_train, x_test, y_test = process_data(data["data"], classification=True)

            start_time = time.time()
            model.fit(x_train, y_train, process_data=False) # model.fit(x_train, y_train)
            metrics = model.evaluate(x_test, y_test)
            end_time = time.time()
            f.write(','.join([dataset] + [f'{val:5.4f}' for val in [metrics["Test ACC"], metrics["Test BACC"], metrics["Test CE"], metrics["Test AUC"], end_time-start_time]]))
            f.write('\n')
            f.flush()